In [ ]:
import os
import matplotlib.pyplot as plt

from data import DIV2K
from model.edsr import edsr
from train import EdsrTrainer

%matplotlib inline

In [ ]:
# Number of residual blocks
depth = 16

numFilters = 64

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [ ]:
# Number of residual blocks
depth = 32

numFilters = 256

# Super-resolution factor
scale = 4

# Downgrade operator
downgrade = 'bicubic'

In [ ]:
# Location of model weights (needed for demo)
weights_path = "weights/weights-edsr-16-x4-fine-tuned.h5"

In [ ]:
# Location of model weights (needed for demo)
weights_path = "weights/weights-edsr-32-x4-fine-tuned.h5"

In [ ]:
model = edsr(scale=scale, num_res_blocks=depth, num_filters=numFilters)
model.load_weights(weights_path)

In [ ]:
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import Mean

loss = MeanAbsoluteError()

loss_mean = Mean()
psnr_mean = Mean()
psnr_mean2 = Mean()
ssim_mean = Mean()

In [ ]:
from model import resolve_single
from utils import load_image
import numpy as np
import tensorflow as tf
import math

def evaluate(start, end):
    base = ".div2k/images/"
    hrPath = base + "DIV2K_valid_HR/0"
    lrPath = base + "DIV2K_valid_LR_bicubic/X4/0"
    for i in range(start, end+1):
        name = str(800+i)
        lr = load_image(lrPath+name+"x4.png")
        sr = resolve_single(model, lr)
        hr = load_image(hrPath+name+".png")
        psnrValue = tf.image.psnr(hr, sr, max_val=255)
        psnr_mean(psnrValue)
        sr = tf.convert_to_tensor(sr)
        hr = tf.convert_to_tensor(hr)
        sr = tf.cast(sr, tf.float32)
        hr = tf.cast(hr, tf.float32)
        lossValue = loss(hr, sr)
        loss_mean(lossValue)
        psnr2 = 10 * math.log10(255*255/lossValue)
        psnr_mean2(psnr2)
        ssimValue = tf.image.ssim(hr, sr, max_val=255)
        ssim_mean(ssimValue)
        print(i)

In [ ]:
evaluate(1, 100)
print("MAE = " + str(loss_mean.result()))
print("PSNR = " + str(psnr_mean.result()))
print("PSNR2 = " + str(psnr_mean2.result()))
print("SSIM = " + str(ssim_mean.result()))

In [ ]:
evaluate(1, 100)
print("MAE = " + str(loss_mean.result()))
print("PSNR = " + str(psnr_mean.result()))
print("PSNR2 = " + str(psnr_mean2.result()))
print("SSIM = " + str(ssim_mean.result()))

In [ ]:
from model import resolve_single
from utils import load_image
import numpy as np
from PIL import Image

def resolve_and_save(lr_image_path, hr_image_path):
    lr = load_image(lr_image_path)
    
    sr = resolve_single(model, lr)
    
    sr = np.array(sr)
    sr = Image.fromarray(sr)
    sr.save(hr_image_path)

In [ ]:
lrPath = ".div2k/images/DIV2K_train_LR_bicubic/X4/0022x4.png"
hrPath = "images/HighRes/0022EDSR16.png"
resolve_and_save(lrPath, hrPath)